In [1]:
import pandas as pd

In [2]:
df = pd.read_stata('lfp_equations_for_python_not_sq.dta')
df

,female,age_group,_b_time,_b_cons
0,0.0,Age (5-year bands): 15-19,-0.967739,1977.860596
1,0.0,Age (5-year bands): 20-24,-0.467796,1013.768005
2,0.0,Age (5-year bands): 25-29,-0.022005,138.650925
3,0.0,Age (5-year bands): 30-34,0.026482,44.793465
4,0.0,Age (5-year bands): 35-39,0.017493,63.419106
5,0.0,Age (5-year bands): 40-44,0.024565,48.941051
6,0.0,Age (5-year bands): 45-49,-0.001132,100.156639
7,0.0,Age (5-year bands): 50-54,0.046355,1.696030
8,0.0,Age (5-year bands): 55-59,-0.037136,164.650345
9,0.0,Age (5-year bands): 60-64,0.008578,54.271637


In [3]:
df_main = pd.read_stata('37382-0001-Data.dta')

In [4]:
df_main[['female']] = pd.get_dummies(df_main.RO3).iloc[:,1:]
df_main['education'] = df_main['ED6'].cat.codes
df_main['education'] = pd.to_numeric(df_main['education'], errors='coerce')
df_main['age'] = df_main['RO5'].replace(" ", 0)
df_main['age'] = pd.to_numeric(df_main['age'], errors='coerce')

In [5]:
df_sub1 = pd.DataFrame(df_main,
                   columns=(
                        'age',
                        'education',
                       'female'
                   ))
df_sub1

,age,education,female
0,50,0,0
1,45,0,1
2,22,8,0
3,15,9,1
4,13,7,0
...,...,...,...
420306,26,5,0
420307,39,15,0
420308,34,15,1
420309,7,2,0


In [6]:
_b_cons = df['_b_cons']
_b_time = df['_b_time']

def exp_pemp(year, i) :
    p = _b_cons[i] + _b_time[i] * year
    return p / 100
exp_pemp(2000, 10)

0.45990333080291745

In [7]:
def prob_emp1(year, age, sex) :
    if sex == 0 :
        if age <= 14 :
            return 0
        elif age <= 19 :
            return exp_pemp(year, 0)
        elif age <= 24 :
            return exp_pemp(year, 1)
        elif age <= 29 :
            return exp_pemp(year, 2)
        elif age <= 34 :
            return exp_pemp(year, 3)
        elif age <= 39 :
            return exp_pemp(year, 4)
        elif age <= 44 :
            return exp_pemp(year, 5)
        elif age <= 49 :
            return exp_pemp(year, 6)
        elif age <= 54 :
            return exp_pemp(year, 7)
        elif age <= 59 :
            return exp_pemp(year, 8)
        elif age <= 64 :
            return exp_pemp(year, 9)
        elif age > 64 :
            return exp_pemp(year, 10)
        else:
            raise Exception("An error has occured with the age")
    elif sex == 1 :
        if age <= 14 :
            return 0
        elif age <= 19 :
            return exp_pemp(year, 11)
        elif age <= 24 :
            return exp_pemp(year, 12)
        elif age <= 29 :
            return exp_pemp(year, 13)
        elif age <= 34 :
            return exp_pemp(year, 14)
        elif age <= 39 :
            return exp_pemp(year, 15)
        elif age <= 44 :
            return exp_pemp(year, 16)
        elif age <= 49 :
            return exp_pemp(year, 17)
        elif age <= 54 :
            return exp_pemp(year, 18)
        elif age <= 59 :
            return exp_pemp(year, 19)
        elif age <= 64 :
            return exp_pemp(year, 20)
        elif age > 64 :
            return exp_pemp(year, 21)
        else:
            raise Exception("An error has occured with the age")
    else:
        raise Exception("An error has occured with sex")
        

In [8]:
def exper(age, educ, sex) :
    potential = age - (educ + 4)
    exp = 0
    for yr in range(potential) :
#         print(2011 - potential + yr)
#         print(yr + educ + 4)
        pemp = prob_emp1((2011 - potential) + yr , yr + educ + 4 , sex)
#         print(pemp)
        exp += pemp
#         print(exp)
    return exp


In [9]:
exper = df_sub1.apply( lambda row : exper(row.age, row.education, row.female ) , axis=1)
df_sub1['exper'] = exper
df_sub1

,age,education,female,exper
0,50,0,0,31.833929
1,45,0,1,10.641273
2,22,8,0,3.303444
3,15,9,1,0.000000
4,13,7,0,0.000000
...,...,...,...,...
420306,26,5,0,6.711665
420307,39,15,0,18.128584
420308,34,15,1,4.519902
420309,7,2,0,0.000000


In [10]:
df_sub1['IDPERSON'] = df_main['IDPERSON']
df_sub1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 420311 entries, 0 to 420310
Data columns (total 5 columns):
age          420311 non-null int64
education    420311 non-null int8
female       420311 non-null uint8
exper        420311 non-null float64
IDPERSON     420311 non-null object
dtypes: float64(1), int64(1), int8(1), object(1), uint8(1)
memory usage: 13.6+ MB


In [11]:
df_sub1 = df_sub1.rename({
    'age' : 'merged_age',
    'education' : 'merged_education',
    'female' : "merged_female"
}, axis=1)

In [12]:
df_sub1.to_stata('expected_experience2_not_sq.dta')